# Foundations of Data Science (GDW) 2023



# Exercise III: Markov Chains & PageRank

In the lecture, we have already taken a look at *PageRank*, an algorithm used by Google for their well-known search engine. Let us take a step back and study Markov chains first.

## Part 1: Markov Chains

Markov chains are a tool for observing stochastic processes that evolve over time. 

**Definition** A discrete Markov chain is a sequence of random variables $X_0, X_1, X_2,\dots$ that satisfy the Markov property, meaning that for all time steps $t$, the probability distribution for $X_{t+1}$ depends only on $X_t$. 


For better understanding, let us take a loot at an example.


Hint: For graph algorithms, python provides the package `networkx`. 
With this package, you may easily
create a directed graph $G = (V, E)$ by its edgelist $E$ as follows (we added some matplotlib functions
to plot the graph):

We consider a graph $G$ with 3 nodes. For every edge $e \in E$, the transition matrix describes the possibility of tranistioning between the connected nodes. If there is no edge between two nodes, the probability is 0.

The graph shows the relations of different weather conditions given a current state. The probabilities can be seen in the transition matrix $T$ below:
$T = \begin{bmatrix}
    2/3 & 1/3 & 0 \\
    1/2 & 0 & 1/2 \\
    1/3 & 1/3 & 1/3
    \end{bmatrix}$

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

states = [(0, 0), (0, 1), (1, 0), (2, 2), (2, 0), (2, 1), (1, 2)]
labels = {0: "sunny", 1: "cloudy", 2: "rainy"}
T = [[2/3, 1/3, 0], [1/2, 0, 1/2], [1/3, 1/3, 1/3]]

G = nx.MultiDiGraph()
G.clear()
G.add_edges_from(states)
#G.add_edges_from([(1, 2), (3,1), (3,4)])
pos = nx.spring_layout(G)
nx.draw(G,pos,with_labels = False)
nx.draw_networkx_labels(G, pos, labels, font_size=22)
plt.show() # display

### Task 1.1
Let $\rho_t$ be the probability distribution on the states of day $t$. You can calculate the distribution for the next day by performing the vector-matrix-multiplication $\rho_{t+1} = \rho_t \cdot T$.

Assuming today's (day 0) weather is *sunny*, calculate the possibilities up to day 5, either by hand or programmatically.

In [ ]:
# Write your code/calculations here

## Part 2: Stepwise PageRank

Let us now consider the following graph.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
G = nx.DiGraph()
G.clear()
G.add_edges_from([(1, 2), (2, 3), (3,4), (4,3), (4,5), (5,1),(5,2),(5,3),(5,4), (4,2)])
#G.add_edges_from([(1, 2), (3,1), (3,4)])
pos = nx.spring_layout(G)
nx.draw(G,pos,with_labels = True)
plt.show() # display

We can use graphs to compute their PageRank.

*Parts of the following code are obtained from 
https://michaelnielsen.org/blog/using-your-laptop-to-compute-pagerank-for-millions-of-webpages/*

In [ ]:
import networkx as nx
import numpy as np
import numpy
import random

class web:
  def __init__(self,n):
    self.size = n
    self.in_links = {}
    self.number_out_links = {}
    self.dangling_pages = {}
    for j in range(n):
      self.in_links[j] = []
      self.number_out_links[j] = 0
      self.dangling_pages[j] = True

def web_from_nx(G):
    edges = list(G.out_edges)
    nodes = list(G.nodes)
    n = len(nodes)
    g = web(n)
    a = np.matrix(G.out_edges)
    for k in range(len(list(G.nodes))):
        adj = a[np.where(a[:,1] == (k+1))[0],0].tolist()
        values=list()
        for node in adj:
            values.append(node[0]-1)
        g.in_links[k] = values
        for j in values: 
            if g.number_out_links[j] == 0: g.dangling_pages.pop(j)
            g.number_out_links[j] += 1
    return g        

def step(g,p,c=0.85):
  n = g.size
  v = numpy.matrix(numpy.zeros((n,1)))
  inner_product = sum([p[j] for j in g.dangling_pages.keys()])
  for j in range(n):
    v[j] = c*sum([p[k]/g.number_out_links[k] 
    for k in g.in_links[j]]) + c*inner_product/n + (1-c)/n
  return v/numpy.sum(v)    

def pagerank(g,c=0.85,tolerance=0.00001):
  n = g.size
  p = numpy.matrix(numpy.ones((n,1)))/n
  iteration = 1
  change = 2
  while change > tolerance:
    #print ("Iteration: %s" % iteration)
    new_p = step(g,p,c)
    change = numpy.sum(numpy.abs(p-new_p))
    #print ("Change in l1 norm: %s" % change)
    p = new_p
    iteration += 1
  return p

G = nx.DiGraph()
G.clear()

#G.add_edges_from([(1,2),(2,1),(3,1),(3,4),(4,3),(4,1)]) # spider trap
#G.add_edges_from([(1,2),(1,3),(2,1),(2,3)]) # dead end

G.add_edges_from([(1, 2), (2, 3), (3,4), (4,3), (4,5), (5,1),(5,2),(5,3),(5,4),(4,2)])
g = web_from_nx(G)

pr = pagerank(g,.99,0.0001)
pr.transpose()

In [ ]:
def paretosample(n,power=2.0):
  # Returns a sample from a truncated Pareto distribution
  # with probability mass function p(l) proportional to
  # 1/l^power.  The distribution is truncated at l = n.'''
  m = n+1
  while m > n: m = numpy.random.zipf(power)
  return m

def random_web(n=1000,power=2.0):
  g = web(n)
  for k in range(n):
    lk = paretosample(n+1,power)-1
    values = random.sample(range(n),lk)
    g.in_links[k] = values
    for j in values: 
      if g.number_out_links[j] == 0: g.dangling_pages.pop(j)
      g.number_out_links[j] += 1
  return g

g = random_web(10000,2.0) 
pr = pagerank(g,.99,0.0001)
pr.transpose()

### Task 2.1
Based on the listing above apply pagerank to *dead ends* and observe the behaviour.
In order to do that, you have to create your own graph first.

In [ ]:
# write your code here

### Task 2.2
Now, do the same thing for *spider traps*:

In [ ]:
# write your code here

## Part 3: PageRank with Matrices 
One option to store matrices in python is using nested lists. Another one are matrices using the `numpy` package.
A matrix is a two-dimensional data structure where numbers are arranged into rows and columns. 
For example, using nested lists:

```python
A = [[1, 4, 5], 
    [-5, 8, 9]]
```

or `numpy` arrays:

In [ ]:
import numpy as np

A = np.array([[3, 6, 7], [5, -3, 0]])
B = np.array([[1, 1], [2, 1], [3, -3]])

D = A.dot(B) 
print(D)
C = np.matmul(A,B) 
print(C)
A_prime = A.transpose() # transpose of A
print(A_prime)

vector = np.array([1,1,1,1,1]).transpose()
vector_normalized = vector/np.linalg.norm(vector,ord=1)

Visit https://docs.scipy.org/doc/numpy-1.15.1/user/quickstart.html for a tutorial on matrix operations using numpy.

## Task 3.1
Given the numpy operators above, implement a matrix notation of pagerank using numpy from an adjacency matrix given as
```python
A=np.array([[0,0,0,0,.25],
            [1,0,0,0,.25],
            [0,1,0,.5,.25],
            [0,0,1,0,.25],
            [0,0,0,.5,0]])
```

In [ ]:
# write your code here